In [ ]:
import urllib.parse
import urllib.request
import pandas as pd
import io

In [38]:
def get_uniprot_id(ensembl_ids):
    url = 'https://www.uniprot.org/uploadlists/'

    params = {
    'from': 'ENSEMBL_ID',
    'to': 'ACC',
    'format': 'tab',
    'query':  " ".join(ensembl_ids)
    }

    data = urllib.parse.urlencode(params)
    data = data.encode('utf-8')
    req = urllib.request.Request(url, data)
    with urllib.request.urlopen(req) as f:
        response = f.read()
        res_df = pd.read_csv(io.StringIO(response.decode('utf-8')), sep="\t")
        return res_df

In [34]:
get_uniprot_id(["ENSG00000243485","ENSG00000274890"])

Empty DataFrame
Columns: [From, To]
Index: []


In [21]:
import gtfparse
def get_genome_annotations(gtf_handle, protein_coding=True):
    """ Parsing and optional filtering of gtf genome annotation file into pd.DataFrame
    """
    gtf_df = gtfparse.read_gtf(gtf_handle)

    if protein_coding:
        # Select only protein coding genes from the gtf
        gtf_df = gtf_df.loc[(gtf_df["gene_type"] == "protein_coding")]

    return gtf_df

In [22]:
gtf_df = get_genome_annotations("/home/oxana/projects/str_database/data/genome_anntotation/gencode.v22.annotation.gtf", protein_coding=True)

INFO:root:Extracted GTF attributes: ['gene_id', 'gene_type', 'gene_status', 'gene_name', 'level', 'havana_gene', 'transcript_id', 'transcript_type', 'transcript_status', 'transcript_name', 'tag', 'transcript_support_level', 'havana_transcript', 'exon_number', 'exon_id', 'ont', 'protein_id', 'ccdsid']


In [37]:
ensembl_ids = []

for index, row in gtf_df.loc[(gtf_df["feature"] == "gene")].iterrows():
    ensembl_id = row["gene_id"].split(".")[0]  # emsebl gene id without version number
    ensembl_ids.append(ensembl_id)
print(len(ensembl_ids))        

19814


In [39]:
uniprot_map = get_uniprot_id(ensembl_ids)

In [42]:
len((uniprot_map["From"]))

75124

In [31]:
only_genes_df = gtf_df[(gtf_df["feature"] == "gene")]

In [35]:
only_genes_df["uniprot_id"] = get_uniprot_id(ensembl_ids)

                  From          To
0      ENSG00000186092  A0A2U3U0J3
1      ENSG00000187634      Q96NU1
2      ENSG00000187634  A0A087WU74
3      ENSG00000187634  A0A087WX24
4      ENSG00000187634  A0A087WXB3
...                ...         ...
75119  ENSG00000212907      P03901
75120  ENSG00000198886      P03905
75121  ENSG00000198786      P03915
75122  ENSG00000198695      P03923
75123  ENSG00000198727      P00156

[75124 rows x 2 columns]


ValueError: Length of values (75124) does not match length of index (19814)